In [ ]:
!pip install emoji
!pip install scikit-learn==1.0.1

     |████████████████████████████████| 170 kB 5.2 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=4cd2f490046729e52897e562b43d12b9b24126e251ff88f8d1e24b285bb1ae96
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji


# Import libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string, re, requests, csv
from google.colab import drive
from wordcloud import WordCloud
from gensim.corpora import WikiCorpus

from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

In [ ]:
from nltk import word_tokenize, sent_tokenize
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Load dataset

In [ ]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')
train.head()

,text,label,username,likes,datetime
0,🔥,negative,lanyardsemarang,0,2020-11-02
1,Naik Turun maju can5ik 😂😂😂😂,neutral,hadi_soeparno,1,2021-03-29
2,Prosentase tingkat kematian karena covid 19 ut...,neutral,heryadisaputro,0,2020-09-22
3,Rapid test/ test swab PCR Di DKK/dinkes Semara...,negative,airlangga15,0,2020-05-29
4,"Min, area pedurungan bs dilakukan dmn,",neutral,fauzanabell,0,2021-06-22


# EDA

## Wordcloud

In [ ]:
# positive comments before preprocessing
# data_pos = train[train['label'] == 'positive']

# all_text = ' '.join(word for word in data_pos['text'])
# wordcloud = WordCloud(colormap='Greens', width=1000, height=1000, mode='RGBA', background_color='white').generate(all_text)
# plt.figure(figsize=(20,10))
# plt.imshow(wordcloud, interpolation='bilinear')
# plt.axis("off")
# plt.margins(x=0, y=0)
# plt.show()

In [ ]:
# negative comments before preprocessing
# data_neg = train[train['label'] == 'negative']

# all_text = ' '.join(word for word in data_neg['text'])
# wordcloud = WordCloud(colormap='Reds', width=1000, height=1000, mode='RGBA', background_color='white').generate(all_text)
# plt.figure(figsize=(20,10))
# plt.imshow(wordcloud, interpolation='bilinear')
# plt.axis("off")
# plt.margins(x=0, y=0)
# plt.show()

In [ ]:
# neutral comments before preprocessing
# data_neut = train[train['label'] == 'neutral']

# all_text = ' '.join(word for word in data_neut['text'])
# wordcloud = WordCloud(colormap='Blues', width=1000, height=1000, mode='RGBA', background_color='white').generate(all_text)
# plt.figure(figsize=(20,10))
# plt.imshow(wordcloud, interpolation='bilinear')
# plt.axis("off")
# plt.margins(x=0, y=0)
# plt.show()

In [ ]:
# value counts
train['label'].value_counts()

neutral     2926
negative    2775
positive    2266
Name: label, dtype: int64

# Preprocess

In [ ]:
# train_text = train['text']
# test_text = test['text']

In [ ]:
# # CLEANSING
# def cleansing(data):

#     # lowercasing
#     data = data.lower()

#     # remove punctuation
#     punct = string.punctuation
#     translator = str.maketrans(punct, ' '*len(punct))
#     data = data.translate(translator)

#     # remove ASCII dan unicode
#     # data = data.encode('ascii', 'ignore').decode('utf-8')
#     # data = re.sub(r'[^\x00-\x7f]',r'', data)
    
#     # remove newline
#     data = data.replace('\n', ' ')

#     # remove digit
#     pattern = r'[0-9]'
#     data = re.sub(pattern, '', data)

#     # remove extra space
#     data = ' '.join(data.split())
    
#     return data

# import sys
# # REMOVE EMOJI
# # def remove_emoji(data):
# #     emoji_pattern = re.compile("["
# #                            u"\U0001F600-\U0001F64F"  # emoticons
# #                            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
# #                            u"\U0001F680-\U0001F6FF"  # transport & map symbols
# #                            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
# #                            u"\U00002702-\U000027B0"
# #                            u"\U000024C2-\U0001F251"
# #                            "]+", flags=re.UNICODE)
# #     return emoji_pattern.sub(r' ', data)

# # CONVERT EMOJIS
# import emoji
# import functools
# import operator
# import re

# df_emoji = pd.read_csv('/content/emoji_to_text.csv')
# UNICODE_EMO = {row['emoji']:row['makna'] for idx,row in df_emoji.iterrows()}
# def convert_emojis(text):
#     # split emojis
#     em_split_emoji = emoji.get_emoji_regexp().split(text)
#     em_split_whitespace = [substr.split() for substr in em_split_emoji]
#     em_split = functools.reduce(operator.concat, em_split_whitespace)
#     text = ' '.join(em_split)

#     # convert emojis
#     for emot in UNICODE_EMO:
#         text = re.sub(r'('+emot+')', "_".join(UNICODE_EMO[emot].replace(",","").replace(":","").split()), text)
#     return text.lower()

# # CONSTRUCT KAMUS ALAY
# text_path1 = 'https://raw.githubusercontent.com/ramaprakoso/analisis-sentimen/master/kamus/kbba.txt'
# text_path2 = 'https://raw.githubusercontent.com/nasalsabila/kamus-alay/master/colloquial-indonesian-lexicon.csv'
# kamus_alay1 = pd.read_csv(text_path1, delimiter="\t", header=None, names=['slang', 'formal'])
# kamus_alay2 = pd.read_csv(text_path2)
# kamus_alay = pd.concat([kamus_alay1, kamus_alay2[['slang', 'formal']]]).reset_index(drop=True)

# dict_alay = dict()
# for index, row in kamus_alay.iterrows():
#     dict_alay[row['slang']] = row['formal']

# # NORMALIZE COLLOQUIAL/ALAY
# def normalize_text(data):
#   word_tokens = word_tokenize(data)
#   result = [dict_alay.get(w,w) for w in word_tokens]
#   return ' '.join(result)


# # CONSTRUCT STOPWORDS
# rama_stopword = "https://raw.githubusercontent.com/ramaprakoso/analisis-sentimen/master/kamus/stopword.txt"
# yutomo_stopword = "https://raw.githubusercontent.com/yasirutomo/python-sentianalysis-id/master/data/feature_list/stopwordsID.txt"
# fpmipa_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-list/fpmipa-stopwords.txt"
# sastrawi_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-list/sastrawi-stopwords.txt"
# aliakbar_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-list/aliakbars-bilp.txt"
# pebahasa_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-list/pebbie-pebahasa.txt"
# elang_stopword = "https://raw.githubusercontent.com/onlyphantom/elangdev/master/elang/word2vec/utils/stopwords-id.txt"
# nltk_stopword = stopwords.words('indonesian')

# path_stopwords = [rama_stopword, yutomo_stopword, fpmipa_stopword, sastrawi_stopword, 
#                   aliakbar_stopword, pebahasa_stopword, elang_stopword]

# # CUSTOM STOPWORDS
# other = '''
# admin mimin min minkes kalo nya username
# '''

# # gabungkan stopwords
# stopwords_l = nltk_stopword
# for path in path_stopwords:
#     response = requests.get(path)
#     stopwords_l += response.text.split('\n')

# st_words = set(stopwords_l)
# other_stopword = set(other.split())

# stop_words = st_words | other_stopword

# # REMOVE STOPWORDS
# def remove_stopword(text, stop_words=stop_words):
#     word_tokens = word_tokenize(text)
#     filtered_sentence = [w for w in word_tokens if not w in stop_words]
#     return ' '.join(filtered_sentence)

In [ ]:
# # full pipeline preprocess
# def preprocess(data):
#     data = cleansing(data)
#     # data = remove_emoji(data)
#     data = convert_emojis(data)
#     data = normalize_text(data)
#     data = remove_stopword(data)
#     return data

In [ ]:
# # rename username to @username
# pattern = "(?:@)([A-Za-z0-9_](?:(?:[A-Za-z0-9_]|(?:\.(?!\.))){0,28}(?:[A-Za-z0-9_]))?)"
# train_text = train_text.apply(lambda x: re.sub(pattern, "@username", x))
# test_text = test_text.apply(lambda x: re.sub(pattern, "@username", x))

# # preprocess
# train_text = train_text.apply(lambda x: preprocess(x))
# test_text = test_text.apply(lambda x: preprocess(x))

In [ ]:
# train_text.head()

In [ ]:
# import random
# id = random.randint(0,len(train_text))

# train['text'][id]

In [ ]:
# text = '''Makasih infonya.....ayo jgn lengah......tetap smangat... pandemi belom usai...

# ...tetap ikuti protokol kesehatan dengan baik....🙏🙏🙏'''
# text = cleansing(text)
# text = convert_emojis(text)
# text = normalize_text(text)
# text = remove_stopword(text)
# text

In [ ]:
# dict_alay.get('jgn','a')

# Feature extraction (Word2Vec Fasttext)

In [ ]:
# # dowload pre-trained word2vec fasttext indonesia
# ! wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.id.300.bin.gz
# # unzip
# ! gunzip cc.id.300.bin.gz

In [ ]:
# from gensim.models import KeyedVectors
# from gensim.models.wrappers import FastText

# # load pre-trained word2vec fasttext
# word2vec = FastText.load_fasttext_format('cc.id.300.bin')

In [ ]:
# # check vocab pada pre-trained model
# vocab = word2vec.wv.vocab
# words = word2vec.wv.index2word

# print(f'vocabulary length: {len(vocab)}')
# print(f'top 20 words in word2vec:\n{words[:20]}')

In [ ]:
# get word vector 'makan'
# word2vec['makan']

In [ ]:
# # tokenize text
# def tokenize(sentence):
#     return word_tokenize(sentence)

# train_text = train_text.apply(lambda x: tokenize(x))
# test_text = test_text.apply(lambda x: tokenize(x))

In [ ]:
# train_text[1]

In [ ]:
# # vectorize
# VOCABULARY = word2vec.wv.vocab
# def vectorize(tokens):
#     word_vec = []
#     for w in tokens:
#         if w in VOCABULARY:
#             # get word vector from pre-trained word2vec fasttext
#             word_vec.append(word2vec[w])
#     return word_vec

# def avg_vectorize(tokens):
#     sum_vec = np.zeros(300)
#     word_count = 0

#     for w in tokens:
#         if w in VOCABULARY:
#             # word vector from pre-trained word2vec fasttext and add vector
#             sum_vec += word2vec[w]
#             word_count += 1
#     return sum_vec if word_count==0 else sum_vec/word_count

In [ ]:
# # check maximum token and count
# max_size_token = 0
# count = 0
# max_len_choosen = 50

# for tok in train_text:
#     # find max token
#     if len(tok) > max_size_token: max_size_token = len(tok)
#     # count
#     if len(tok) > max_len_choosen: count += 1

# print(f'Maximum length token: {max_size_token}')
# print(f'With MAX_LEN {max_len_choosen}, there are/is {count} token/s')

In [ ]:
# # padding
# MAX_LEN = 50

# def add_padding(word_vec):
#     if len(word_vec) < MAX_LEN:
#         pad_count = MAX_LEN - len(word_vec)
#         return word_vec + [np.array([0]*300)]*pad_count
#     else:
#         return word_vec[:MAX_LEN]

In [ ]:
# # feature extraction
# def extract_feature(data, ndim=3):
#     if ndim == 3:
#         features = vectorize(data)
#         features = add_padding(features)
#     elif ndim == 2:
#         features = avg_vectorize(data)
#     return np.array(features)

In [ ]:
# extract feature
# train_features = np.array([extract_feature(text, ndim=2) for text in train_text])
# test_features = np.array([extract_feature(text, ndim=2) for text in test_text])

# train_features.shape, test_features.shape

In [ ]:
# export
# np.save("train_rf_wordvec.npy", train_features)
# np.save("test_rf_wordvec.npy", test_features)

In [ ]:
path_train = "/content/drive/MyDrive/5. Research & Development/Kepengurusan 2021-2022/1 - Research Instagram DKK Semarang/Code and Model/train_rf_wordvec.npy"
path_test = "/content/drive/MyDrive/5. Research & Development/Kepengurusan 2021-2022/1 - Research Instagram DKK Semarang/Code and Model/test_rf_wordvec.npy"
train_features = np.load(path_train)
test_features = np.load(path_test)

train_features.shape, test_features.shape

((7967, 300), (1992, 300))

NOTE:
- Metode neural network/deep learning seperti LSTM, CNN bisa menerima feature input 3 dimensi.
- Metode SVM tidak bisa menerima input 3 dimensi, sehingga fitur harus dirubah menjadi 2 dimensi. 

# Random Forest

In [ ]:
# mapping label
mapper = {'neutral':0, 'positive':1, 'negative':2}
train_y = train['label'].map(mapper)
test_y = test['label'].map(mapper)

# train_y

In [ ]:
# from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(train_features, train_y)

RandomForestClassifier()

In [ ]:
%%time
# cross-val score

scores = cross_val_score(clf, train_features, train_y, cv=5)
print(scores)

[0.66562108 0.67001255 0.68298807 0.69052103 0.66729441]
CPU times: user 45.1 s, sys: 90 ms, total: 45.2 s
Wall time: 47.3 s


In [ ]:
clf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [ ]:
# predict
y_pred = clf.predict(test_features)

In [ ]:
# accuracy
from sklearn.metrics import accuracy_score

accuracy_score(test_y, y_pred)

0.6822289156626506

In [ ]:
# from joblib import dump, load

# dump(clf, 'word2vec_svc.joblib') 

In [ ]:
%%time
# Randomized Search
# references: 
# https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

from sklearn.model_selection import RandomizedSearchCV
import numpy as np
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
            #    'max_features': max_features,
            #    'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

clf = RandomForestClassifier()
clf_random = RandomizedSearchCV(estimator=clf, 
                                param_distributions=random_grid, 
                                n_iter=10, 
                                cv=5, 
                                random_state=2021)
# Fit the random search model
clf_random.fit(train_features, train_y)

CPU times: user 1h 57min 29s, sys: 9.39 s, total: 1h 57min 38s
Wall time: 1h 57min 5s


In [ ]:
clf_random.best_params_

{'bootstrap': False,
 'min_samples_leaf': 4,
 'min_samples_split': 10,
 'n_estimators': 2000}

In [ ]:
y_pred = clf_random.predict(test_features)

accuracy = accuracy_score(test_y, y_pred)
print(accuracy)

0.6852409638554217


In [ ]:
%%time

# cross-val score
scores = cross_val_score(clf_random, train_features, train_y, cv=5)
print(scores)

In [ ]:
clf_random.get_params()

{'cv': 5,
 'error_score': nan,
 'estimator': RandomForestClassifier(),
 'estimator__bootstrap': True,
 'estimator__ccp_alpha': 0.0,
 'estimator__class_weight': None,
 'estimator__criterion': 'gini',
 'estimator__max_depth': None,
 'estimator__max_features': 'auto',
 'estimator__max_leaf_nodes': None,
 'estimator__max_samples': None,
 'estimator__min_impurity_decrease': 0.0,
 'estimator__min_samples_leaf': 1,
 'estimator__min_samples_split': 2,
 'estimator__min_weight_fraction_leaf': 0.0,
 'estimator__n_estimators': 100,
 'estimator__n_jobs': None,
 'estimator__oob_score': False,
 'estimator__random_state': None,
 'estimator__verbose': 0,
 'estimator__warm_start': False,
 'n_iter': 10,
 'n_jobs': None,
 'param_distributions': {'bootstrap': [True, False],
  'min_samples_leaf': [1, 2, 4],
  'min_samples_split': [2, 5, 10],
  'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]},
 'pre_dispatch': '2*n_jobs',
 'random_state': 2021,
 'refit': True,
 'return_train_score': 

GridSearchCV

In [ ]:
%%time

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

# params = {'kernel': ['linear', 'rbf', 'poly'],  # 3
#           'C': [1, 0.25, 0.5, 0.75],            # 4
#           'gamma': ['scale', 'auto', 1, 2, 3],  # 5
#           'decision_function_shape': ['ovo','ovr'] # 2
#           # 3 * 4 * 5 *2 * 5cv = 120 * 5 = 600 kombinasi
# }

param_grid = { 
    'n_estimators': [100,200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [2,3,4,5,6],
    'criterion' :['gini', 'entropy']
}

clf = RandomForestClassifier()
cv_test = StratifiedKFold(n_splits=5)
clf_grid = GridSearchCV(clf, param_grid, cv=cv_test)
clf_grid.fit(train_features, train_y)

In [ ]:
# from joblib import dump, load

# # dump(clf_grid, 'word2vec_svc_tuned.joblib')
# # clf_grid = load('word2vec_svc_tuned.joblib')
# clf_grid = load('word2vec_svc.joblib')

In [ ]:
clf_grid.best_params_

{'criterion': 'entropy',
 'max_depth': 6,
 'max_features': 'auto',
 'n_estimators': 500}

In [ ]:
%%time

# cross-val score
scores = cross_val_score(clf_grid, train_features, train_y, cv=5)
print(scores)

In [ ]:
# accuracy test
y_pred = clf_grid.predict(test_features)

accuracy_score(test_y, y_pred)

In [ ]:
# from joblib import dump, load

# dump(clf_grid, 'word2vec_svc_tuned.joblib') 